In [ ]:
import RPi.GPIO as GPIO
import adafruit_dht
import time
import datetime
import random
import time
from paho.mqtt import client as mqtt_client

In [ ]:
broker = 'broker.emqx.io'
port = 1883
topic = "python/mqtt"
client_id = f'python-mqtt-{random.randint(0, 1000)}'

def connect_mqtt():
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)

    client = mqtt_client.Client(client_id)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client


def publish(client,temp):
    msg_count = 0
    time.sleep(1)
    msg = f"messages: {msg_count}"
    result = client.publish(topic, msg)
    status = result[0]
    if status == 0:
        print(f"Send `{temp}` to topic `{topic}`")
    else:
        print(f"Failed to send message to topic {topic}")
    msg_count += 1


def run(temp):
    publish(client,temp)


In [ ]:
def checkTempValidity(instance):
    try:
        # Print the values to the serial port
        temperature_c = instance.temperature
#         humidity = instance.humidity
        if temperature_c>24:
#             print("Temp: {:.1f} F / {:.1f} C Humidity: {}% ".format(temperature_f, temperature_c, humidity))
            time.sleep(1.0)
            return True
        else:
            return False
         
    except RuntimeError as error: # Errors happen fairly often, DHT's are hard to read, just keep going
        print(error.args[0])
        
    except KeyboardInterrupt:
        print('interrupted!')
        instance.exit()

In [ ]:
# initialize GPIO
GPIO.setwarnings(True)
GPIO.setmode(GPIO.BCM)
# GPIO.setmode(GPIO.BOARD)

temperature_c = 0

instance = adafruit_dht.DHT11(4)
pir_pin = 26
GPIO.setwarnings(False)
GPIO.setup(pir_pin, GPIO.IN)
temp="set"

# mqtt
client = connect_mqtt()
client.loop_start()

channel = 21
GPIO.setup(channel, GPIO.OUT)

def motor_on(pin):
    GPIO.output(pin, GPIO.HIGH)  # Turn motor on

def motor_off(pin):
    GPIO.output(pin, GPIO.LOW)  # Turn motor off

if __name__ == '__main__':
    try:
        while(True):
            temp_status = checkTempValidity(instance)
            motion = GPIO.input(pir_pin)
            if(temp_status and (int(motion) == 1)):
#                 run(temp)
                motor_on(channel)
                time.sleep(1)
                print("Working")
            else:
                motor_off(channel)
                time.sleep(1)
                print("Not working")
        GPIO.cleanup()
    except KeyboardInterrupt:
        GPIO.cleanup()